# Starbucks Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.


### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.



# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


In [371]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# Read in the json files
portfolio = pd.read_json('./data/portfolio.json', orient = 'records', lines = True)
profile = pd.read_json('./data/profile.json', orient = 'records', lines = True)
transcript = pd.read_json('./data/transcript.json', orient = 'records', lines = True)

# Data Exploration

**Portfolio Dataset**

In [372]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [373]:
portfolio.describe()

,reward,difficulty,duration
count,10.000000,10.000000,10.000000
mean,4.200000,7.700000,6.500000
std,3.583915,5.831905,2.321398
min,0.000000,0.000000,3.000000
25%,2.000000,5.000000,5.000000
50%,4.000000,8.500000,7.000000
75%,5.000000,10.000000,7.000000
max,10.000000,20.000000,10.000000


In [374]:
portfolio.shape

(10, 6)

In [375]:
portfolio.isnull().sum()

reward        0
channels      0
difficulty    0
duration      0
offer_type    0
id            0
dtype: int64

In [376]:
portfolio.groupby(by = "offer_type")['id'].count()

offer_type
bogo             4
discount         4
informational    2
Name: id, dtype: int64

In [377]:
portfolio['difficulty'].unique()

array([10,  0,  5, 20,  7])

**Profile Dataset**

In [378]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [379]:
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


In [380]:
profile.shape

(17000, 5)

In [381]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
gender              14825 non-null object
age                 17000 non-null int64
id                  17000 non-null object
became_member_on    17000 non-null int64
income              14825 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


In [382]:
profile['age'].unique()

array([118,  55,  75,  68,  65,  58,  61,  26,  62,  49,  57,  40,  64,
        78,  42,  56,  33,  46,  59,  67,  53,  22,  96,  69,  20,  45,
        54,  39,  41,  79,  66,  29,  44,  63,  36,  76,  77,  30,  51,
        27,  73,  74,  70,  89,  50,  90,  60,  19,  72,  52,  18,  71,
        83,  43,  47,  32,  38,  34,  85,  48,  35,  82,  21,  24,  81,
        25,  37,  23, 100,  28,  84,  80,  87,  86,  94,  31,  88,  95,
        93,  91,  92,  98, 101,  97,  99])

In [383]:
profile['gender'].unique()

array([None, 'F', 'M', 'O'], dtype=object)

In [384]:
profile.isna().sum()

gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64

In [385]:
profile.gender.value_counts()

M    8484
F    6129
O     212
Name: gender, dtype: int64

In [386]:
profile['income'].describe()

count     14825.000000
mean      65404.991568
std       21598.299410
min       30000.000000
25%       49000.000000
50%       64000.000000
75%       80000.000000
max      120000.000000
Name: income, dtype: float64

In [387]:
profile['age'].value_counts()

118    2175
58      408
53      372
51      363
54      359
       ... 
100      12
96        8
98        5
99        5
101       5
Name: age, Length: 85, dtype: int64

In [388]:
profile[profile['age'] == 118].count()

gender                 0
age                 2175
id                  2175
became_member_on    2175
income                 0
dtype: int64

**Transcript Dataset**

In [389]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [390]:
transcript.describe()

,time
count,306534.000000
mean,366.382940
std,200.326314
min,0.000000
25%,186.000000
50%,408.000000
75%,528.000000
max,714.000000


In [391]:
transcript.shape

(306534, 4)

In [392]:
transcript.isnull().sum()

person    0
event     0
value     0
time      0
dtype: int64

In [393]:
transcript['event'].unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

In [394]:
transcript.columns.duplicated().sum()

0

In [395]:
transcript['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [396]:
transcript['time'].unique()

array([  0,   6,  12,  18,  24,  30,  36,  42,  48,  54,  60,  66,  72,
        78,  84,  90,  96, 102, 108, 114, 120, 126, 132, 138, 144, 150,
       156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228,
       234, 240, 246, 252, 258, 264, 270, 276, 282, 288, 294, 300, 306,
       312, 318, 324, 330, 336, 342, 348, 354, 360, 366, 372, 378, 384,
       390, 396, 402, 408, 414, 420, 426, 432, 438, 444, 450, 456, 462,
       468, 474, 480, 486, 492, 498, 504, 510, 516, 522, 528, 534, 540,
       546, 552, 558, 564, 570, 576, 582, 588, 594, 600, 606, 612, 618,
       624, 630, 636, 642, 648, 654, 660, 666, 672, 678, 684, 690, 696,
       702, 708, 714])

In [397]:
df_transaction = transcript[transcript['event'] == "transaction"]

In [398]:
df_transaction

,person,event,value,time
12654,02c083884c7d45b39cc68e1314fec56c,transaction,{'amount': 0.8300000000000001},0
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,{'amount': 34.56},0
12659,54890f68699049c2a04d415abc25e717,transaction,{'amount': 13.23},0
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,{'amount': 19.51},0
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,{'amount': 18.97},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


# Data Preprocessing

**Portfolio Dataset**

In [399]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [400]:
# Rename id to offer_id
portfolio.rename(columns={'id':'offer_id'}, inplace = True)
portfolio.head()

,reward,channels,difficulty,duration,offer_type,offer_id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [401]:
# Convert duration from days to hours
portfolio['duration'] = portfolio['duration'].apply(lambda x: x*24)

# Rename duration to duration_h
portfolio.rename(columns={'duration':'duration_h'},inplace=True)

portfolio.head()

,reward,channels,difficulty,duration_h,offer_type,offer_id
0,10,"[email, mobile, social]",10,168,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,120,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,96,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,168,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,240,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [402]:
# Modify dataframe for one-hot encoding on channels
portfolio['ch_email'] = portfolio['channels'].apply(lambda x: 1 if 'email' in x else 0)
portfolio['ch_mobile'] = portfolio['channels'].apply(lambda x: 1 if 'mobile' in x else 0)
portfolio['ch_social'] = portfolio['channels'].apply(lambda x: 1 if 'social' in x else 0)
portfolio['ch_web'] = portfolio['channels'].apply(lambda x: 1 if 'web' in x else 0)

In [403]:
# Modify encoding for offer types
portfolio.loc[portfolio['offer_type'] == 'bogo', 'offer_type'] = 1
portfolio.loc[portfolio['offer_type'] == 'discount', 'offer_type'] = 2
portfolio.loc[portfolio['offer_type'] == 'informational', 'offer_type'] = 3

In [404]:
# Drop the previous channels columns for the new one-hot encoding
portfolio.drop(labels = 'channels', axis = 1, inplace = True)

In [405]:
portfolio.head()

,reward,difficulty,duration_h,offer_type,offer_id,ch_email,ch_mobile,ch_social,ch_web
0,10,10,168,1,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,10,120,1,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,96,3,3f207df678b143eea3cee63160fa8bed,1,1,0,1
3,5,5,168,1,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1
4,5,20,240,2,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1


In [406]:
# Normalize difficulty and reward columns with MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
x = ['difficulty','reward']
portfolio[x] = scaler.fit_transform(portfolio[x])

In [407]:
portfolio.head()

,reward,difficulty,duration_h,offer_type,offer_id,ch_email,ch_mobile,ch_social,ch_web
0,1.0,0.50,168,1,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,1.0,0.50,120,1,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0.0,0.00,96,3,3f207df678b143eea3cee63160fa8bed,1,1,0,1
3,0.5,0.25,168,1,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1
4,0.5,1.00,240,2,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1


In [408]:
# Transformation for offer ID
labels_offer_id = portfolio['offer_id'].astype('category').cat.categories.tolist()
replace_offer_id = {'offer_id' : {k: v for k, v in zip(labels_offer_id, list(range(1, len(labels_offer_id) + 1)))}}

In [409]:
replace_offer_id

{'offer_id': {'0b1e1539f2cc45b7b9fa7c272da2e1d7': 1,
  '2298d6c36e964ae4a3e7e9706d1fb8c2': 2,
  '2906b810c7d4411798c6938adc9daaa5': 3,
  '3f207df678b143eea3cee63160fa8bed': 4,
  '4d5c57ea9a6940dd891ad53e9dbe8da0': 5,
  '5a8bc65990b245e5a138643cd4eb9837': 6,
  '9b98b8c7a33c4b65b9aebfe6a799e6d9': 7,
  'ae264e3637204a6fb9bb56bc8210ddfd': 8,
  'f19421c1d4aa40978ebb69ca19b0e20d': 9,
  'fafdcd668e3743c1bb461111dcafc2a4': 10}}

In [410]:
portfolio.replace(replace_offer_id, inplace = True)

In [411]:
portfolio.head()

,reward,difficulty,duration_h,offer_type,offer_id,ch_email,ch_mobile,ch_social,ch_web
0,1.0,0.50,168,1,8,1,1,1,0
1,1.0,0.50,120,1,5,1,1,1,1
2,0.0,0.00,96,3,4,1,1,0,1
3,0.5,0.25,168,1,7,1,1,0,1
4,0.5,1.00,240,2,1,1,0,0,1


**Profile Dataset**

In [412]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [413]:
# Rename id to customer_id
profile.rename(columns = {'id':'customer_id'}, inplace = True)

In [414]:
# Rearrange columns
profile = profile.reindex(columns = ['customer_id', 'age', 'became_member_on', 'gender', 'income'])

In [415]:
# Transform customer ID as done with offer ID
labels_cus_id = profile['customer_id'].astype('category').cat.categories.tolist()
replace_cus_id = {'customer_id' : {k: v for k, v in zip(labels_cus_id, list(range(1, len(labels_cus_id) + 1)))}}
profile.replace(replace_cus_id, inplace = True)

In [416]:
profile.head()

,customer_id,age,became_member_on,gender,income
0,6962,118,20170212,None,NaN
1,399,55,20170715,F,112000.0
2,3747,118,20180712,None,NaN
3,7997,75,20170509,F,100000.0
4,10736,118,20170804,None,NaN


In [417]:
# Replace age 118 with NaN
profile['age'] = profile['age'].apply(lambda x: np.nan if x == 118 else x)

In [418]:
# Drop values with NaN in columns: age, income, and gender
profile = profile.dropna()

In [419]:
profile.isnull().sum()

customer_id         0
age                 0
became_member_on    0
gender              0
income              0
dtype: int64

In [420]:
profile['age'].dtype

dtype('float64')

In [421]:
# Change float to int for age and income
profile = profile.astype({"age": int, "income": int})

In [422]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14825 entries, 1 to 16999
Data columns (total 5 columns):
customer_id         14825 non-null int64
age                 14825 non-null int64
became_member_on    14825 non-null int64
gender              14825 non-null object
income              14825 non-null int64
dtypes: int64(4), object(1)
memory usage: 694.9+ KB


In [423]:
x = profile['age'].unique()

In [424]:
# Create new column grouped with age group
profile['age_group'] = pd.cut(profile['age'], bins = [17, 22, 35, 60, 103], labels = ['teenager', 'young-adult', 'adult', 'elderly'])

In [425]:
x = profile['age_group'].astype('category').cat.categories.tolist()

In [426]:
y = {"age_group": {k:v for k, v in zip(x, list(range(1, len(x) + 1)))}}

In [427]:
profile.replace(y, inplace = True)

In [428]:
profile.drop(columns = "age", inplace = True)

In [429]:
profile.head()

,customer_id,became_member_on,gender,income,age_group
1,399,20170715,F,112000,3
3,7997,20170509,F,100000,4
5,15044,20180426,M,70000,4
8,3729,20180209,M,53000,4
12,3060,20171111,M,51000,3


In [430]:
# Create new column income range
profile['income_range'] = pd.cut(profile['income'], bins = [29999, 60000, 90000, 120001], labels = ['average', 'above_average', 'high'])

In [431]:
profile['income_range'].head()

1              high
3              high
5     above_average
8           average
12          average
Name: income_range, dtype: category
Categories (3, object): [average < above_average < high]

In [432]:
profile = profile.drop(columns = "income")

In [433]:
profile.head()

,customer_id,became_member_on,gender,age_group,income_range
1,399,20170715,F,3,high
3,7997,20170509,F,4,high
5,15044,20180426,M,4,above_average
8,3729,20180209,M,4,average
12,3060,20171111,M,3,average


In [434]:
labels = profile['income_range'].astype('category').cat.categories.tolist()

In [435]:
x = {'income_range': {k:v for k, v in zip(labels, list(range(1, len(labels) + 1)))}}

In [436]:
profile.replace(x, inplace = True)

In [437]:
profile.head()

,customer_id,became_member_on,gender,age_group,income_range
1,399,20170715,F,3,3
3,7997,20170509,F,4,3
5,15044,20180426,M,4,2
8,3729,20180209,M,4,1
12,3060,20171111,M,3,1


In [438]:
# Modify gender to 1, 2, and 3
labels = profile['gender'].astype('category').cat.categories.tolist()

In [439]:
x = {'gender': {k:v for k, v in zip(labels, list(range(1, len(labels) + 1)))}}

In [440]:
profile.replace(x,inplace = True)

In [441]:
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format = '%Y%m%d')

In [442]:
profile['became_member_on']

1       2017-07-15
3       2017-05-09
5       2018-04-26
8       2018-02-09
12      2017-11-11
           ...    
16995   2018-06-04
16996   2018-07-13
16997   2017-01-26
16998   2016-03-07
16999   2017-07-22
Name: became_member_on, Length: 14825, dtype: datetime64[ns]

In [443]:
profile['membership_year'] = pd.to_datetime(profile['became_member_on']).dt.year

In [444]:
profile['membership_year']

1        2017
3        2017
5        2018
8        2018
12       2017
         ... 
16995    2018
16996    2018
16997    2017
16998    2016
16999    2017
Name: membership_year, Length: 14825, dtype: int64

In [445]:
import datetime
profile['membership_days'] = datetime.datetime.today().date() - profile['became_member_on'].dt.date

In [446]:
profile['membership_days'] = profile['membership_days'].dt.days

In [447]:
profile.head()

,customer_id,became_member_on,gender,age_group,income_range,membership_year,membership_days
1,399,2017-07-15,1,3,3,2017,1193
3,7997,2017-05-09,1,4,3,2017,1260
5,15044,2018-04-26,2,4,2,2018,908
8,3729,2018-02-09,2,4,1,2018,984
12,3060,2017-11-11,2,3,1,2017,1074


In [448]:
profile['membership_days'].describe()

count    14825.000000
mean      1339.478988
std        419.205158
min        817.000000
25%       1025.000000
50%       1175.000000
75%       1614.000000
max       2640.000000
Name: membership_days, dtype: float64

In [449]:
profile['member_type'] = pd.cut(profile['membership_days'], bins = [665, 1199, 1999, 2488], labels = ['new', 'regular', 'loyal'])

In [450]:
x = profile['member_type'].astype('category').cat.categories.tolist()

In [451]:
y = {'member_type': {k:v for k, v in zip(x, list(range(1, len(x) + 1)))}}

In [452]:
profile.replace(y, inplace = True)

In [453]:
# Convert to int64
profile['member_type'] = profile['member_type'].values.astype(np.int64)

In [454]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14825 entries, 1 to 16999
Data columns (total 8 columns):
customer_id         14825 non-null int64
became_member_on    14825 non-null datetime64[ns]
gender              14825 non-null int64
age_group           14825 non-null int64
income_range        14825 non-null int64
membership_year     14825 non-null int64
membership_days     14825 non-null int64
member_type         14825 non-null int64
dtypes: datetime64[ns](1), int64(7)
memory usage: 1.0 MB


In [455]:
profile.drop(columns = ['membership_days', 'became_member_on'], inplace = True)

In [456]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14825 entries, 1 to 16999
Data columns (total 6 columns):
customer_id        14825 non-null int64
gender             14825 non-null int64
age_group          14825 non-null int64
income_range       14825 non-null int64
membership_year    14825 non-null int64
member_type        14825 non-null int64
dtypes: int64(6)
memory usage: 810.7 KB


**Transcript Dataset**

In [457]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [458]:
transcript['time'].describe()

count    306534.000000
mean        366.382940
std         200.326314
min           0.000000
25%         186.000000
50%         408.000000
75%         528.000000
max         714.000000
Name: time, dtype: float64

In [459]:
# Rename time to time_h
transcript.rename(columns = {'time':'time_h'}, inplace = True) 

In [460]:
# Rename person to customer_id
transcript.rename(columns = {'person': 'customer_id'}, inplace = True)

In [461]:
# Modify customer_id to simple numeric ID
x = transcript['customer_id'].astype('category').cat.categories.tolist()

In [462]:
y = {'customer_id': {k:v for k, v in zip(x, list(range(1, len(x) + 1)))}}

In [463]:
transcript.replace(y, inplace = True)

In [464]:
transcript.head()

,customer_id,event,value,time_h
0,7997,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,10736,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,15044,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,9525,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,6940,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [465]:
# Value column extract values like offer id, amount, reward, offer_id
'''keys=[]
for idx,row in transcript.iterrows():
    for rows in transcript['value']:
        if rows in keys:
            continue
        else:
            keys.append(rows)
keys'''

"keys=[]\nfor idx,row in transcript.iterrows():\n    for rows in transcript['value']:\n        if rows in keys:\n            continue\n        else:\n            keys.append(rows)\nkeys"

In [466]:
transcript['offer_id'] = ''
transcript['amount'] = 0

transcript['reward'] = 0

In [467]:
# Put offer_id amount and reward in respective column
for idx, row in transcript.iterrows():
    for k in row['value']:
        if k == 'offer_id' or k == 'offer id':
            transcript.at[idx, 'offer_id'] = row['value'][k]
        if k == 'amount':
            transcript.at[idx, 'amount'] = row['value'][k]
        if k == 'reward':
            transcript.at[idx, 'reward'] = row['value'][k]

In [468]:
# Put NaN where offer_id is N/A'
transcript['offer_id'] = transcript['offer_id'].apply(lambda x: 'N/A' if x == '' else x) 

In [469]:
transcript.head()

,customer_id,event,value,time_h,offer_id,amount,reward
0,7997,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,0
1,10736,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,0
2,15044,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,2906b810c7d4411798c6938adc9daaa5,0,0
3,9525,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,fafdcd668e3743c1bb461111dcafc2a4,0,0
4,6940,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,4d5c57ea9a6940dd891ad53e9dbe8da0,0,0


In [470]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 7 columns):
customer_id    306534 non-null int64
event          306534 non-null object
value          306534 non-null object
time_h         306534 non-null int64
offer_id       306534 non-null object
amount         306534 non-null int64
reward         306534 non-null int64
dtypes: int64(4), object(3)
memory usage: 16.4+ MB


In [471]:
transcript['offer_id']

0         9b98b8c7a33c4b65b9aebfe6a799e6d9
1         0b1e1539f2cc45b7b9fa7c272da2e1d7
2         2906b810c7d4411798c6938adc9daaa5
3         fafdcd668e3743c1bb461111dcafc2a4
4         4d5c57ea9a6940dd891ad53e9dbe8da0
                        ...               
306529                                 N/A
306530                                 N/A
306531                                 N/A
306532                                 N/A
306533                                 N/A
Name: offer_id, Length: 306534, dtype: object

In [368]:
transcript['event']

0         offer received
1         offer received
2         offer received
3         offer received
4         offer received
               ...      
306529       transaction
306530       transaction
306531       transaction
306532       transaction
306533       transaction
Name: event, Length: 306534, dtype: object

In [472]:
# Drop values with transaction and offer received
transcript['event'] = transcript[transcript['event'] != 'transaction']
transcript['event'] = transcript[transcript['event'] != 'offer received']

In [473]:
transcript['event']

0          7997
1         10736
2         15044
3          9525
4          6940
          ...  
306529    12028
306530     6914
306531    10723
306532     7882
306533    12848
Name: event, Length: 306534, dtype: object

In [474]:
transcript.head()

,customer_id,event,value,time_h,offer_id,amount,reward
0,7997,7997,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,0
1,10736,10736,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,0
2,15044,15044,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,2906b810c7d4411798c6938adc9daaa5,0,0
3,9525,9525,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,fafdcd668e3743c1bb461111dcafc2a4,0,0
4,6940,6940,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,4d5c57ea9a6940dd891ad53e9dbe8da0,0,0


In [475]:
x = transcript['event'].astype('category').cat.categories.tolist()
y = {'event': {k:v  for k, v in zip(x, list(range(1, len(x) + 1)))}}

In [476]:
transcript.head()

,customer_id,event,value,time_h,offer_id,amount,reward
0,7997,7997,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,0
1,10736,10736,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,0
2,15044,15044,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,2906b810c7d4411798c6938adc9daaa5,0,0
3,9525,9525,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,fafdcd668e3743c1bb461111dcafc2a4,0,0
4,6940,6940,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,4d5c57ea9a6940dd891ad53e9dbe8da0,0,0


In [477]:
# Replace offer completed and viewed with 1, and 2
transcript.replace(y, inplace = True)

ValueError: Replacement not allowed with overlapping keys and values

In [263]:
transcript

,customer_id,event,value,time_h,offer_id,amount,reward
0,7997,7997,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,7,0,0
1,10736,10736,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1,0,0
2,15044,15044,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,3,0,0
3,9525,9525,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,11,0,0
4,6940,6940,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,5,0,0
...,...,...,...,...,...,...,...
306529,12028,12028,{'amount': 1.5899999999999999},714,8,1,0
306530,6914,6914,{'amount': 9.53},714,8,9,0
306531,10723,10723,{'amount': 3.61},714,8,3,0
306532,7882,7882,{'amount': 3.5300000000000002},714,8,3,0


In [264]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 7 columns):
customer_id    306534 non-null int64
event          306534 non-null object
value          306534 non-null object
time_h         306534 non-null int64
offer_id       306534 non-null int64
amount         306534 non-null int64
reward         306534 non-null int64
dtypes: int64(5), object(2)
memory usage: 16.4+ MB


In [265]:
transcript['event'] = transcript['event'].fillna(0).astype(int)

In [266]:
transcript.head()

,customer_id,event,value,time_h,offer_id,amount,reward
0,7997,7997,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,7,0,0
1,10736,10736,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1,0,0
2,15044,15044,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,3,0,0
3,9525,9525,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,11,0,0
4,6940,6940,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,5,0,0


In [267]:
transcript.replace(replace_offer_id,inplace=True)

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [528]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 7 columns):
customer_id    306534 non-null int64
event          306534 non-null int64
value          306534 non-null object
time_h         306534 non-null int64
offer_id       167581 non-null float64
amount         138953 non-null float64
reward         33579 non-null float64
dtypes: float64(3), int64(3), object(1)
memory usage: 16.4+ MB


In [529]:
portfolio.rename(columns={'id':'offer_id'},inplace=True)
portfolio.head()

,reward,difficulty,duration_h,offer_type,offer_id,ch_email,ch_mobile,ch_social,ch_web
0,1.0,0.50,168,1,8,1,1,1,0
1,1.0,0.50,120,1,5,1,1,1,1
2,0.0,0.00,96,3,4,1,1,0,1
3,0.5,0.25,168,1,7,1,1,0,1
4,0.5,1.00,240,2,1,1,0,0,1


In [530]:
transcript['offer_id']=transcript['offer_id'].apply(lambda x: 0 if x =='N/A' else x)
transcript['offer_id'].unique()

array([ 7.,  1.,  3., 10.,  5.,  9.,  2.,  4.,  8.,  6., nan])

In [531]:
transcript['offer_id'].unique()

array([ 7.,  1.,  3., 10.,  5.,  9.,  2.,  4.,  8.,  6., nan])

In [532]:
transcript.drop(transcript.index[transcript['event'] == 0],inplace=True)


# Merge datasets to master df 

In [ ]:
#merge all three df
master_df=transcript.merge(portfolio,how="left",on="offer_id")
master_df=master_df.merge(profile,how="left",on="customer_id")
master_df.head()

# Data analysis on master dataset

In [ ]:
from matplotlib import *
import pylab as plt
%matplotlib inline
master_df.info()

In [ ]:
master_df.dropna(inplace=True)

In [ ]:
master_df.info()

In [ ]:
master_df.columns

In [ ]:
master_df.rename(columns={'reward_x':'reward'},inplace=True)

In [ ]:
#finding correlation between labels
corr=master_df.corr()

In [ ]:
import seaborn as sns
sns.heatmap(corr)

# Data Modeling

In [ ]:
#important features for training obtained from corr
X = master_df[['time_h','offer_id','amount','reward','difficulty','duration_h','offer_type','gender','age_group','income_range', 'member_type']]
Y = master_df['event']

In [ ]:
#split train test
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [ ]:
#function for checking accuracy
def predict_score(model):
    pred = model.predict(X_test)
    
    # Calculate the absolute errors
    errors = abs(pred - y_test)
    
    # Calculate mean absolute percentage error
    mean_APE = 100 * (errors / y_test)
    accuracy = 100 - np.mean(mean_APE)
    
    return round(accuracy, 4)

**1) Logistic Regression**

In [ ]:
logreg = LogisticRegression()

logreg.fit(X_train, y_train)
print(f'Accuracy of Logistic regression classifier on training set: {round(logreg.score(X_train, y_train)*100,2)}%.')
print(f'Prediction Accuracy: {predict_score(logreg)}%')

**2) Random Forest Classifier**

In [ ]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

rf.fit(X_train, y_train)
print(f'Accuracy of SVM classifier on training set: {round(rf.score(X_train, y_train)*100,2)}%.')
print(f'Prediction Accuracy: {predict_score(rf)}%')

**3) Support Vector Machine**

In [ ]:
svm = SVC(gamma = 'auto')

svm.fit(X_train, y_train)
print(f'Accuracy of SVM classifier on training set: {round(svm.score(X_train, y_train)*100,2)}%.')
print(f'Prediction Accuracy: {predict_score(svm)}%')


**4) Decision Tree**

In [ ]:
dt = DecisionTreeClassifier()

dt.fit(X_train, y_train)
print(f'Accuracy of Decision Tree classifier on training set: {round(dt.score(X_train, y_train)*100,2)}%.')
print(f'Prediction Accuracy: {predict_score(dt)}%')

**5) Naive Bayes**

In [ ]:
gnb = GaussianNB() 
gnb.fit(X_train, y_train) 
print(f'Accuracy of SVM classifier on training set: {round(gnb.score(X_train, y_train)*100,2)}%.')
print(f'Prediction Accuracy: {predict_score(gnb)}%')

**6) K nearest neighbors**

In [ ]:
knn = KNeighborsClassifier()

knn.fit(X_train, y_train)
print(f'Accuracy of K-NN classifier on training set: {round(knn.score(X_train, y_train)*100,2)}%.')
print(f'Prediction Accuracy: {predict_score(knn)}%')

# Model evaluation

In [ ]:
models = [svm, dt, gnb, knn, rf, logreg]
model_names = [type(n).__name__ for n in models]
training_accuracy = [x.score(X_train, y_train)*100 for x in models]
predection_accuracy = [predict_score(y) for y in models]

In [ ]:
results = [training_accuracy, predection_accuracy]
results_df = pd.DataFrame(results, columns = model_names, index=['Training Accuracy', 'Predicting Accuracy']) 
results_df

# Model Refinement

These models are giving very good accuracy, however Naive Bayes and Logistic regression is not upto the mark. 
In addition to that, I think SVC DT and RF are overfitted because it is very close to 100. That's why I chose KNN model for further refinement with GridSearchCV (sklearn pipeline).

# Questions based on Data visualization

In [ ]:
# reconverting the values of the following features from numerical values to its original categorical values.
master_df['event'] = master_df['event'].map({1: 'Completed', 2: 'Viewed'})
master_df['offer_type'] = master_df['offer_type'].map({1: 'BOGO', 2: 'Discount', 3: 'Informational'})
master_df['income_range'] = master_df['income_range'].map({1: 'Average', 2: 'Above-Average', 3:'High'})
master_df['age_group'] = master_df['age_group'].map({1: 'teenager', 2: 'young-adult', 3:'adult', 4:'elderly'})

**What is the common offer each age group ( teenagers, young-adults, adults and elderly)?**

In [ ]:
plt.figure(figsize=(14, 6))
g = sns.countplot(x="age_group", hue="offer_type", data=master_df)
plt.title('Most Popular Offers to Each Age Group')
plt.ylabel('Total')
plt.xlabel('Age Group')
xlabels = ['teenager','young-adult','adult','elderly']
g.set_xticklabels(xlabels)
plt.xticks(rotation = 0)
plt.legend(title='Offer Type')
plt.show();

The most common offer type among all age groups is the BOGO , followed by the Discount Offers. Whereas, the least common offer to be sent is the informational offers. I believe that BOGO offers are more attractive compared to other offers provided by Starbucks.

**Based on the demographic data of the customers who gets the highest income range , males or females?**

In [ ]:
plt.figure(figsize=(14, 6))
g = sns.countplot(x="gender", hue="income_range", data= master_df[master_df["gender"] != 3])
plt.title('Income Range vs Gender')
plt.ylabel('Income Range')
xlabels = ['Female', 'Male']
g.set_xticklabels(xlabels)
plt.xlabel('Gender')
plt.xticks(rotation = 0)
plt.show();

Customers with High income (Above 90,000) are mostly female customers. Whereas, Average Income(30,000 - 60,000) customers are mostly males.

**How many new members Starbucks got each year?**

In [ ]:
plt.figure(figsize=(16, 6))
sns.countplot(master_df['membership_year'])
plt.title('Number of Profiles In Each Year')
plt.ylabel('Number of Profiles')
plt.xlabel('Year')
plt.xticks()
plt.show();

2017 was the best year for Starbucks in terms of the number of new members.

**Which type of promotions(offers) each gender likes?**

In [ ]:
plt.figure(figsize=(14, 6))
g = sns.countplot(x='gender', hue="offer_type", data= master_df[master_df["gender"] != 3])
plt.title('Most Popular Offers to Each Gender')
plt.ylabel('Total')
plt.xlabel('Gender')
xlabels = ['Female', 'Male']
g.set_xticklabels(xlabels)
plt.legend(title='Offer Type')
plt.show();

Both genders liked BOGO and lest liked is Infomational offer.

**What is average time to complete offer?**

In [ ]:
tran_avg_len = master_df.groupby(['customer_id', 'offer_id'])['time_h'].mean().reset_index()
tran_avg_len['time_h'].mean(), tran_avg_len['time_h'].mean() / 24

The mean time it takes a customer to complete an offer is less than 16 days (372 hours).

**From all the offers the customers viewed , how many offers they completed?**

In [ ]:
plt.figure(figsize=(14, 6))
g = sns.countplot(x='gender', hue="event", data= master_df[master_df["gender"] != 3])
plt.title('Most Popular Offers to Each Gender')
plt.ylabel('Total')
plt.xlabel('Gender')
xlabels = ['Female', 'Male']
g.set_xticklabels(xlabels)
plt.legend(title='Offer Type')
plt.show();

Females responded highly in terms of completing offer compared to male.

# Improvement

I believe I got to a point where I had very good results when it comes to the prediction model. I understood the data provided by Starbucks very well.

I believe that the idea of improving the data collection or fixing issues related with the missing data or NaNs would be really helpful. In my opinion, I think that we can get great insights from this data set and great prediction models might be also built to solve problems statemetns associated with this data set . Examples for these model are the following :

Building a model that can predict which kind of offers to be sent to which customer?
Building a model that can predict which customers would buy any way ( regardless if there is an offer or not)

# Reflection

I really enjoyed working on this project which allowed me to polish my skills in Data Preprocessing and Modeling. The most difficult part in my opinion was understanding data. 

The Preprocessing Step was the longest and most challanging part to complete. The Dataset itself is inspiring. Honestly,Everything was amazing.